In [193]:
from sqlanalyzer import column_parser
import pandas as pd
import sqlparse
import re

In [333]:
from sqlanalyzer import column_parser
import pandas as pd
import sqlparse
import re
import json


def get_joins_pos(query_list):

    pos_delete, pos_where = [len(query_list)-1], len(query_list)
    pos_join = []
    for i, line in enumerate(query_list):
        if line.startswith('ORDER') or line.startswith('GROUP'):
            pos_delete.append(i)
        if line.startswith('FROM') and len(line.split(' ')) > 1:
            pos_join.append(i)
        elif line.startswith('FROM') and len(line.split(' ')) == 1:
            pos_join.append(i+1)
        if line.startswith('WHERE'):
            pos_where = i
        if line.startswith('LEFT JOIN') or line.startswith('INNER JOIN') or line.startswith('FULL OUTER JOIN'):
            pos_join.append(i+1)

    if min(pos_delete) == len(query_list)-1:
        pos_join.append(min(pos_delete))
    else:
        pass

    return pos_join, pos_where


def get_alias_pos(query_list, pos_join, pos_where):

    pos_join_list = iter(pos_join)
    next(pos_join_list)
    alias_pos = []

    if query_list[pos_join[0]].startswith('FROM'):
        alias_pos.append(pos_join[0])

    for i in range(len(pos_join)-1):
        if i < len(pos_join)-2 and pos_join[i] < pos_where:
            end_pos = next(pos_join_list)-1
            alias_pos.append(end_pos-1)

        elif pos_join[-1] >= pos_where:
            end_pos = next(pos_join_list)-1
            alias_pos.append(pos_where - 1)
        
        else:
            end_pos = pos_join[-1]
            alias_pos.append(pos_where-1)

    alias_pos = sorted(list(set(alias_pos)))

    return alias_pos


def parse_sub_query(query_list, sub_query_pos):

    sub_query = {}
    keep = []
    for _, sub_pos in enumerate(sub_query_pos):
        alias = query_list[sub_pos[1]]
        query = query_list[sub_pos[0]: sub_pos[1]]

        try:
            alias_list_rev = alias.split(' ')[::-1]
            if alias_list_rev[0][-1] != ')':
                alias_index = alias_list_rev.index('ON')
                alias = alias_list_rev[alias_index+1]

                if alias_list_rev[alias_index+2] == 'AS':
                    keep.append(' '.join(alias_list_rev[:alias_index+3][::-1]))
                    del alias_list_rev[:alias_index+3]

                else:
                    keep.append(' '.join(alias_list_rev[:alias_index+2][::-1]))
                    del alias_list_rev[:alias_index+2]

                query.append(' '.join(alias_list_rev[::-1]).rstrip(r'\)').lstrip(' '))

            else:
                alias_list_rev[0] = alias_list_rev[0].rstrip(r'\)')
                alias = 'no alias'
                query.append(' '.join(alias_list_rev[::-1]))

        except:
            query.append(' '.join(alias.split(' ')[:-1]).rstrip(r'\)').lstrip(' '))
            alias = alias.split(' ')[-1]

        trans_query = ' '.join(query).lstrip(r' \(').lstrip(' FROM')
    
        if trans_query == '':
            sub_query = {}
        else:
            sub_query[alias] = trans_query
        
    return sub_query, keep


def delevel(query_list):

    sub_query = {}
    pos_join, pos_where = get_joins_pos(query_list)
    alias_pos = get_alias_pos(query_list, pos_join, pos_where)
    sub_query_pos = list(zip(pos_join, alias_pos))
    sub_query, keep = parse_sub_query(query_list, sub_query_pos)
    main_query_pos = main_query(query_list, sub_query_pos)
    if main_query_pos != []:
        sub_query['main'] = '\n'.join([query_list[p] for p in main_query_pos])
    sub_query['main'] = sub_query['main'] + ' ' + '\n'.join(keep)
    
    return sub_query


def has_child(formatted_query):
    
    count = 0
    if len(delevel(formatted_query.split('\n')).keys()) > 1:
        for _,v in delevel(formatted_query.split('\n')).items():
            if v != {}: count += 1
            
    return count != 0


def within(num, rng):
    if num >= min(rng) and num <= max(rng): return 1
    else: return 0

    
def main_query(query_list, sub_query_pos):
    l = []
    for i in range(len(query_list)): 
        count = 0
        for pair in sorted(sub_query_pos):
            count += within(i, pair)
        if count == 0:
            l.append(i)
    return l
        
    
def clean_dict(query_dict):
    for k,v in query_dict.items(): 
        if isinstance(v, dict) and len(v.keys()) == 1 and 'main' in v.keys():
            query_dict[k] = v['main']
    return query_dict


In [365]:
query = """SELECT *
   FROM
     (SELECT a.*,
             b.*,
             c.*,
             d.*
      FROM
        (SELECT DISTINCT anonymous_id,
                         user_id
         FROM mapbox_customer_data.segment_identifies
         WHERE dt >= '2018-07-01'
           AND anonymous_id IS NOT NULL
           AND user_id IS NOT NULL ) a
      LEFT JOIN
        (SELECT id,
                email,
                created
         FROM mapbox_customer_data.accounts
         WHERE cast(dt AS DATE) = CURRENT_DATE - INTERVAL '1' DAY ) b ON a.user_id = b.id
      LEFT JOIN
        (SELECT anonymous_id AS anon_id_ad,
                context_campaign_name,
                min(TIMESTAMP) AS min_exposure
         FROM mapbox_customer_data.segment_pages
         WHERE dt >= '2018-07-01'
           AND context_campaign_name IS NOT NULL
         GROUP BY 1,
                  2) c ON a.anonymous_id = c.anon_id_ad
      LEFT JOIN
        (SELECT DISTINCT anonymous_id AS anon_id_event,
                         original_timestamp,
                         event,
                         context_traits_email
         FROM mapbox_customer_data.segment_tracks
         WHERE dt >= '2018-07-01'
           AND event LIKE 'submitted_%form'
           AND context_traits_email IS NOT NULL ) d ON a.anonymous_id = d.anon_id_event
    LEFT JOIN
        (SELECT sfdc_accounts.platform, sfdc_accounts.mobile_os, sfdc_accounts.service_metadata,
sfdc_cases.account, sfdc_cases.num_requests, sfdc_cases.owner, sfdc_accounts.user_id
FROM sfdc.accounts sfdc_accounts
LEFT JOIN 
(SELECT MAX(dt) FROM 
    (SELECT dt 
    FROM sfdc.oppty 
    LEFT JOIN (SELECT MAX(dt) FROM (SELECT DISTINCT dt FROM sfdc.owner AS sfdc_owner) AS dt_owner ON sfdc_oppty.dt = sfdc_cases.dt)
    LEFT JOIN (SELECT dt FROM sfdc.cases) sfdc_cases ON sfdc_oppty.dt = sfdc_cases.dt) )
AS sfdc_cases_oppty ON sfdc_cases_oppty.dt = sfdc_accounts.dt
WHERE sfdc_cases_oppty.dt > '2020-04-03' AND sfdc_cases_oppty.dt < '2020-05-04' ORDER BY 1 GROUP BY 3 LIMIT 20
        ) e ON e.user_id = a.user_id
        )
   WHERE context_campaign_name IS NOT NULL 
"""

In [340]:
formatter = column_parser.Parser(query)
formatted_query = formatter.format_query(query)
query_list_0 = formatted_query.split('\n')
query_dict = {}
sub_query = delevel(query_list_0)
query_dict = sub_query

In [341]:
query_dict

{'no alias': "SELECT a.*,           b.*,           c.*,           d.*    FROM      (SELECT DISTINCT anonymous_id,                       user_id       FROM mapbox_customer_data.segment_identifies       WHERE dt >= '2018-07-01'         AND anonymous_id IS NOT NULL         AND user_id IS NOT NULL ) a    LEFT JOIN      (SELECT id,              email,              created       FROM mapbox_customer_data.accounts       WHERE cast(dt AS DATE) = CURRENT_DATE - INTERVAL '1' DAY ) b ON a.user_id = b.id    LEFT JOIN      (SELECT anonymous_id AS anon_id_ad,              context_campaign_name,              min(TIMESTAMP) AS min_exposure       FROM mapbox_customer_data.segment_pages       WHERE dt >= '2018-07-01'         AND context_campaign_name IS NOT NULL       GROUP BY 1,                2) c ON a.anonymous_id = c.anon_id_ad    LEFT JOIN      (SELECT DISTINCT anonymous_id AS anon_id_event,                       original_timestamp,                       event,                       context_traits_

In [342]:
query_dict['no alias']

"SELECT a.*,           b.*,           c.*,           d.*    FROM      (SELECT DISTINCT anonymous_id,                       user_id       FROM mapbox_customer_data.segment_identifies       WHERE dt >= '2018-07-01'         AND anonymous_id IS NOT NULL         AND user_id IS NOT NULL ) a    LEFT JOIN      (SELECT id,              email,              created       FROM mapbox_customer_data.accounts       WHERE cast(dt AS DATE) = CURRENT_DATE - INTERVAL '1' DAY ) b ON a.user_id = b.id    LEFT JOIN      (SELECT anonymous_id AS anon_id_ad,              context_campaign_name,              min(TIMESTAMP) AS min_exposure       FROM mapbox_customer_data.segment_pages       WHERE dt >= '2018-07-01'         AND context_campaign_name IS NOT NULL       GROUP BY 1,                2) c ON a.anonymous_id = c.anon_id_ad    LEFT JOIN      (SELECT DISTINCT anonymous_id AS anon_id_event,                       original_timestamp,                       event,                       context_traits_email       F

## delevel_query

## level 0 -- level 1 -- level 2 -- level 3 -- level 4 -- level 5

In [366]:
formatter = column_parser.Parser(query)
formatted_query = formatter.format_query(query)
query_list_0 = formatted_query.split('\n')
query_dict = {}
sub_query = delevel(query_list_0)
query_dict = sub_query

for alias, query in sub_query.items():

    formatter = column_parser.Parser(query)
    formatted_query = formatter.format_query(query)
    query_list = formatted_query.split('\n')

    if has_child(formatted_query) and alias != 'main':
        sub_query_dict = delevel(query_list)
        query_dict[alias] = sub_query_dict
        query_dict = clean_dict(query_dict)

        for alias2, query2 in sub_query_dict.items():
            formatter2 = column_parser.Parser(query2)
            formatted_query2 = formatter2.format_query(query2)
            query_list2 = formatted_query2.split('\n')
            
            if has_child(formatted_query2) and alias2 != 'main':
                sub_query_dict2 = delevel(query_list2)
                sub_query_dict[alias2] = sub_query_dict2
                
                
                for alias3, query3 in sub_query_dict2.items():
                    formatter3 = column_parser.Parser(query3)
                    formatted_query3 = formatter3.format_query(query3)
                    query_list3 = formatted_query3.split('\n')
                    
                    if has_child(formatted_query3) and alias3 != 'main':
                        sub_query_dict3 = delevel(query_list3)
                        sub_query_dict2[alias3] = sub_query_dict3
                        
                        for alias4, query4 in sub_query_dict3.items():
                            formatter4 = column_parser.Parser(query4)
                            formatted_query4 = formatter4.format_query(query4)
                            query_list4 = formatted_query4.split('\n')
                            
                            if has_child(formatted_query4) and alias4 != 'main':
                                sub_query_dict4 = delevel(query_list4)
                                sub_query_dict3[alias4] = sub_query_dict4
                            else:
                                pass
                        query_dict[alias][alias2][alias3] = sub_query_dict3
                        query_dict = clean_dict(query_dict)
                    else:
                        pass
                    query_dict[alias][alias2] = sub_query_dict2
                    query_dict = clean_dict(query_dict)
            else:
                pass
            query_dict[alias] = sub_query_dict
            query_dict = clean_dict(query_dict)
    else:
        pass


In [394]:
query_dict

{'no alias': {'a': "SELECT DISTINCT anonymous_id,                    user_id    FROM mapbox_customer_data.segment_identifies    WHERE dt >= '2018-07-01'      AND anonymous_id IS NOT NULL AND user_id IS NOT NULL ",
  'b': "SELECT id,           email,           created    FROM mapbox_customer_data.accounts WHERE cast(dt AS DATE) = CURRENT_DATE - INTERVAL '1' DAY ",
  'c': "SELECT anonymous_id AS anon_id_ad,           context_campaign_name,           min(TIMESTAMP) AS min_exposure    FROM mapbox_customer_data.segment_pages    WHERE dt >= '2018-07-01'      AND context_campaign_name IS NOT NULL    GROUP BY 1, 2",
  'd': "SELECT DISTINCT anonymous_id AS anon_id_event,                    original_timestamp,                    event,                    context_traits_email    FROM mapbox_customer_data.segment_tracks    WHERE dt >= '2018-07-01'      AND event LIKE 'submitted_%form' AND context_traits_email IS NOT NULL ",
  'e': {'sfdc_accounts': 'sfdc.accounts',
   'sfdc_cases_oppty': {'no alia

In [367]:
print(alias, '\n', json.dumps(query_dict, indent=2), '\n\n\n')

main 
 {
  "no alias": {
    "a": "SELECT DISTINCT anonymous_id,                    user_id    FROM mapbox_customer_data.segment_identifies    WHERE dt >= '2018-07-01'      AND anonymous_id IS NOT NULL AND user_id IS NOT NULL ",
    "b": "SELECT id,           email,           created    FROM mapbox_customer_data.accounts WHERE cast(dt AS DATE) = CURRENT_DATE - INTERVAL '1' DAY ",
    "c": "SELECT anonymous_id AS anon_id_ad,           context_campaign_name,           min(TIMESTAMP) AS min_exposure    FROM mapbox_customer_data.segment_pages    WHERE dt >= '2018-07-01'      AND context_campaign_name IS NOT NULL    GROUP BY 1, 2",
    "d": "SELECT DISTINCT anonymous_id AS anon_id_event,                    original_timestamp,                    event,                    context_traits_email    FROM mapbox_customer_data.segment_tracks    WHERE dt >= '2018-07-01'      AND event LIKE 'submitted_%form' AND context_traits_email IS NOT NULL ",
    "e": {
      "sfdc_accounts": "sfdc.accounts",
  

In [369]:
query_dict.keys()

dict_keys(['no alias', 'main'])

In [371]:
query_dict['main']

'SELECT *\nFROM\nWHERE context_campaign_name IS NOT NULL '

In [372]:
query_dict['no alias'].keys()

dict_keys(['a', 'b', 'c', 'd', 'e', 'main'])

In [392]:
query_dict['no alias']['main']

'SELECT a.*,\n       b.*,\n       c.*,\n       d.*\nFROM\nLEFT JOIN\nLEFT JOIN\nLEFT JOIN\nLEFT JOIN b ON a.user_id = b.id\nc ON a.anonymous_id = c.anon_id_ad\nd ON a.anonymous_id = d.anon_id_event\ne ON e.user_id = a.user_id'

In [373]:
query_dict['no alias']['a']

"SELECT DISTINCT anonymous_id,                    user_id    FROM mapbox_customer_data.segment_identifies    WHERE dt >= '2018-07-01'      AND anonymous_id IS NOT NULL AND user_id IS NOT NULL "

In [374]:
query_dict['no alias']['b']

"SELECT id,           email,           created    FROM mapbox_customer_data.accounts WHERE cast(dt AS DATE) = CURRENT_DATE - INTERVAL '1' DAY "

In [375]:
query_dict['no alias']['c']

"SELECT anonymous_id AS anon_id_ad,           context_campaign_name,           min(TIMESTAMP) AS min_exposure    FROM mapbox_customer_data.segment_pages    WHERE dt >= '2018-07-01'      AND context_campaign_name IS NOT NULL    GROUP BY 1, 2"

In [376]:
query_dict['no alias']['d']

"SELECT DISTINCT anonymous_id AS anon_id_event,                    original_timestamp,                    event,                    context_traits_email    FROM mapbox_customer_data.segment_tracks    WHERE dt >= '2018-07-01'      AND event LIKE 'submitted_%form' AND context_traits_email IS NOT NULL "

In [378]:
query_dict['no alias']['e']

{'sfdc_accounts': 'sfdc.accounts',
 'sfdc_cases_oppty': {'no alias': {'dt_owner': 'SELECT DISTINCT dt FROM sfdc.owner AS sfdc_owner',
   'main': 'SELECT MAX(dt)\nFROM AS dt_owner ON sfdc_oppty.dt = sfdc_cases.dt'},
  'sfdc_cases': 'SELECT dt FROM sfdc.cases',
  'main': 'SELECT MAX(dt)\nSELECT dt\nLEFT JOIN\nLEFT JOIN sfdc_cases ON sfdc_oppty.dt = sfdc_cases.dt'},
 'main': "SELECT sfdc_accounts.platform,\n       sfdc_accounts.mobile_os,\n       sfdc_accounts.service_metadata,\n       sfdc_cases.account,\n       sfdc_cases.num_requests,\n       sfdc_cases.owner,\n       sfdc_accounts.user_id\nLEFT JOIN\nWHERE sfdc_cases_oppty.dt > '2020-04-03'\n  AND sfdc_cases_oppty.dt < '2020-05-04'\nORDER BY 1\nGROUP BY 3\nLIMIT 20 AS sfdc_cases_oppty ON sfdc_cases_oppty.dt = sfdc_accounts.dt"}

In [380]:
query_dict['no alias']['e']['main']

"SELECT sfdc_accounts.platform,\n       sfdc_accounts.mobile_os,\n       sfdc_accounts.service_metadata,\n       sfdc_cases.account,\n       sfdc_cases.num_requests,\n       sfdc_cases.owner,\n       sfdc_accounts.user_id\nLEFT JOIN\nWHERE sfdc_cases_oppty.dt > '2020-04-03'\n  AND sfdc_cases_oppty.dt < '2020-05-04'\nORDER BY 1\nGROUP BY 3\nLIMIT 20 AS sfdc_cases_oppty ON sfdc_cases_oppty.dt = sfdc_accounts.dt"

In [381]:
query_dict['no alias']['e']['sfdc_accounts']

'sfdc.accounts'

In [382]:
query_dict['no alias']['e']['sfdc_cases_oppty']

{'no alias': {'dt_owner': 'SELECT DISTINCT dt FROM sfdc.owner AS sfdc_owner',
  'main': 'SELECT MAX(dt)\nFROM AS dt_owner ON sfdc_oppty.dt = sfdc_cases.dt'},
 'sfdc_cases': 'SELECT dt FROM sfdc.cases',
 'main': 'SELECT MAX(dt)\nSELECT dt\nLEFT JOIN\nLEFT JOIN sfdc_cases ON sfdc_oppty.dt = sfdc_cases.dt'}

In [384]:
query_dict['no alias']['e']['sfdc_cases_oppty']['main']

'SELECT MAX(dt)\nSELECT dt\nLEFT JOIN\nLEFT JOIN sfdc_cases ON sfdc_oppty.dt = sfdc_cases.dt'

In [385]:
query_dict['no alias']['e']['sfdc_cases_oppty']['no alias']

{'dt_owner': 'SELECT DISTINCT dt FROM sfdc.owner AS sfdc_owner',
 'main': 'SELECT MAX(dt)\nFROM AS dt_owner ON sfdc_oppty.dt = sfdc_cases.dt'}

In [386]:
query_dict['no alias']['e']['sfdc_cases_oppty']['sfdc_cases']

'SELECT dt FROM sfdc.cases'

In [389]:
query_dict['no alias']['e']['sfdc_cases_oppty']['no alias']['dt_owner']

'SELECT DISTINCT dt FROM sfdc.owner AS sfdc_owner'

In [391]:
query_dict['no alias']['e']['sfdc_cases_oppty']['no alias']['main']

'SELECT MAX(dt)\nFROM AS dt_owner ON sfdc_oppty.dt = sfdc_cases.dt'